Name = Mahyar Jahani nasab



In [7]:
!git clone https://github.com/huggingface/transformers



Cloning into 'transformers'...
remote: Enumerating objects: 139170, done.
remote: Counting objects: 100% (1162/1162), done.
remote: Compressing objects: 100% (470/470), done.
remote: Total 139170 (delta 724), reused 961 (delta 597), pack-reused 138008
Receiving objects: 100% (139170/139170), 137.54 MiB | 26.25 MiB/s, done.
Resolving deltas: 100% (104189/104189), done.


In [8]:
!git apply my-patch.patch

my-patch.patch:57: trailing whitespace.
        
my-patch.patch:72: trailing whitespace.
        
my-patch.patch:80: trailing whitespace.
        
my-patch.patch:81: trailing whitespace.
        


In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
!pip install -e /content/transformers



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/transformers
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for transformers (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.29.0.dev0-0.editable-py3-none-any.whl size=36133 sha256=621fd674be0c8cac7e093a2b86be4a61ae94a5cef691e6373cba90f9738ee1d2
  Stored in directory: /tmp/pip-ephem-wheel-cache-kx5k2116/wheels/7c/35/80/e946b22a081210c6642e607ed65b2a5b9a4d9259695ee2caf5
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.28.1
    Uninstalling transformers-4.28.1:
      Successfully uninstalled transformers-4.28.1


In [2]:
!wget https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

--2023-05-04 05:54:42--  https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  29.6MB/s    in 2.7s    

2023-05-04 05:54:45 (29.6 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [3]:
!tar -zxvf aclImdb_v1.tar.gz

Streaming output truncated to the last 5000 lines.
aclImdb/train/unsup/44983_0.txt
aclImdb/train/unsup/44982_0.txt
aclImdb/train/unsup/44981_0.txt
aclImdb/train/unsup/44980_0.txt
aclImdb/train/unsup/44979_0.txt
aclImdb/train/unsup/44978_0.txt
aclImdb/train/unsup/44977_0.txt
aclImdb/train/unsup/44976_0.txt
aclImdb/train/unsup/44975_0.txt
aclImdb/train/unsup/44974_0.txt
aclImdb/train/unsup/44973_0.txt
aclImdb/train/unsup/44972_0.txt
aclImdb/train/unsup/44971_0.txt
aclImdb/train/unsup/44970_0.txt
aclImdb/train/unsup/44969_0.txt
aclImdb/train/unsup/44968_0.txt
aclImdb/train/unsup/44967_0.txt
aclImdb/train/unsup/44966_0.txt
aclImdb/train/unsup/44965_0.txt
aclImdb/train/unsup/44964_0.txt
aclImdb/train/unsup/44963_0.txt
aclImdb/train/unsup/44962_0.txt
aclImdb/train/unsup/44961_0.txt
aclImdb/train/unsup/44960_0.txt
aclImdb/train/unsup/44959_0.txt
aclImdb/train/unsup/44958_0.txt
aclImdb/train/unsup/44957_0.txt
aclImdb/train/unsup/44956_0.txt
aclImdb/train/unsup/44955_0.txt
aclImdb/train/unsup/4

In [2]:
import os
import re
import numpy as np
import pandas as pd
from transformers import AutoConfig, AutoModelForMaskedLM, AutoTokenizer,RobertaTokenizer
from transformers import RobertaConfig
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AutoTokenizer
import torch



device = 'cuda'


In [3]:

from transformers import RobertaForSequenceClassification
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=1).to(device)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.encoder.layer.0.output.adapt

In [4]:
model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [5]:
for name, param in model.named_parameters():
    if 'adapter' in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

In [6]:
for name, param in model.named_parameters():
    print(name, param.requires_grad)

roberta.embeddings.word_embeddings.weight False
roberta.embeddings.position_embeddings.weight False
roberta.embeddings.token_type_embeddings.weight False
roberta.embeddings.LayerNorm.weight False
roberta.embeddings.LayerNorm.bias False
roberta.encoder.layer.0.attention.self.query.weight False
roberta.encoder.layer.0.attention.self.query.bias False
roberta.encoder.layer.0.attention.self.key.weight False
roberta.encoder.layer.0.attention.self.key.bias False
roberta.encoder.layer.0.attention.self.value.weight False
roberta.encoder.layer.0.attention.self.value.bias False
roberta.encoder.layer.0.attention.output.dense.weight False
roberta.encoder.layer.0.attention.output.dense.bias False
roberta.encoder.layer.0.attention.output.LayerNorm.weight False
roberta.encoder.layer.0.attention.output.LayerNorm.bias False
roberta.encoder.layer.0.attention.output.adapter.linear1.weight True
roberta.encoder.layer.0.attention.output.adapter.linear1.bias True
roberta.encoder.layer.0.attention.output.adapt

In [7]:
def load_train_test_imdb_data(data_dir):
    data = {}
    for split in ["train", "test"]:
        data[split] = []
        for sentiment in ["neg", "pos"]:
            score = 1 if sentiment == "pos" else 0

            path = os.path.join(data_dir, split, sentiment)
            file_names = os.listdir(path)
            for f_name in file_names:
                with open(os.path.join(path, f_name), "r") as f:
                    review = f.read()
                    data[split].append([review, score])

    np.random.shuffle(data["train"])        
    data["train"] = pd.DataFrame(data["train"],
                                 columns=['text', 'sentiment'])
    print(data["train"])
    np.random.shuffle(data["test"])
    data["test"] = pd.DataFrame(data["test"],
                                columns=['text', 'sentiment'])
    print(data["test"])
    return data["train"], data["test"]

In [8]:
train_data, test_data = load_train_test_imdb_data(
    data_dir="aclImdb/")

                                                    text  sentiment
0      This is a movie that has a lot of things that ...          1
1      Let's cut to the chase: this movie is softcore...          0
2      This has to be one of the 5 worst movies ever ...          0
3      this, is NOT one of those films it is one of t...          0
4      I thought How The Grinch Stole Christmas was a...          1
...                                                  ...        ...
24995  I went to see this movie (actually I went to s...          0
24996  A documentary without a loss for words... ever...          1
24997  Kurt Russell is so believable and the action s...          1
24998  Bruce Almighty is the best Jim Carrey work sin...          1
24999  Casting bone to pick: David Jannsen was 38 pla...          0

[25000 rows x 2 columns]
                                                    text  sentiment
0      Talk about rubbish! I can't think of one good ...          0
1      I watched a mov

In [9]:
maximum = 512
tokenizer = AutoTokenizer.from_pretrained('roberta-base')
config = AutoConfig.from_pretrained("roberta-base")

class Data_preprocessor(torch.utils.data.Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.comment_text = dataframe["text"].tolist()
        self.targets = dataframe["sentiment"].tolist()
        self.max_len = max_len

    def __len__(self):
        return len(self.comment_text)

    def __getitem__(self, index):
        comment_text = str(self.comment_text[index])
        comment_text = " ".join(comment_text.split())

        inputs = self.tokenizer.encode_plus(
            comment_text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_token_type_ids=True,
            return_tensors='pt'
        )

        ids = inputs['input_ids'].squeeze(0).to(device, dtype = torch.long)
        mask = inputs['attention_mask'].squeeze(0).to(device, dtype = torch.long)
        token_type_ids = inputs['token_type_ids'].squeeze(0).to(device, dtype = torch.long)
        target = torch.tensor(self.targets[index], dtype=torch.long).to(device)

        return {
            'ids': ids,
            'mask': mask,
            'token_type_ids': token_type_ids,
            'targets': target
        }

In [10]:
test_data[:25]

,text,sentiment
0,Talk about rubbish! I can't think of one good ...,0
1,"I watched a movie called Dark Talon, dated 197...",0
2,"Proud as i am of being a Dutchman, i'm truly s...",0
3,The Man with the Golden Arm was one of the fir...,1
4,What's the most violent movie of all time? Ram...,1
5,Accepted is a 2006 teen flick that stars the g...,1
6,This was the first feature film for just about...,0
7,This is one of the best lesbian films i have e...,1
8,"I consider Saboteur as Hitchcock's first ""Amer...",1
9,Hollywood movies since the 1930s have treated ...,1


In [11]:
training_set = Data_preprocessor(train_data, tokenizer, maximum)
testing_set = Data_preprocessor(test_data[:2500], tokenizer, maximum)
train_dataloader = DataLoader(training_set,batch_size=5)
testing_loader = DataLoader(testing_set)

In [12]:
criterion =torch.nn.BCEWithLogitsLoss()  
LEARNING_RATE=1e-5
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)
for epoch in range(2):
  model.train()
  average_loss=0
  for i,data in enumerate(train_dataloader, 0):
      ids = data['ids']
      mask = data['mask']
      token_type_ids = data['token_type_ids']
      targets = data['targets']
      outputs = model(ids, mask, token_type_ids)
      optimizer.zero_grad()
      loss = criterion(outputs[0],targets.view(-1,1).to(device, dtype = torch.float))
      average_loss+=loss
      loss.backward()
      optimizer.step()
  print(f'Epoch: {epoch}, Loss:  {average_loss/(i+1)}')


Epoch: 0, Loss:  0.3506506681442261
Epoch: 1, Loss:  0.18642163276672363


In [13]:
model.eval()
targets_=[]
outputs_=[]
sigmoid_v=torch.nn.Sigmoid()
with torch.no_grad():
    for i,data in enumerate(testing_loader, 0):
      ids = data['ids']
      mask = data['mask']
      token_type_ids = data['token_type_ids']
      targets = data['targets']
      output = model(ids, mask, token_type_ids)
      targets_.extend(targets.cpu().detach().numpy().tolist())
      outputs_.extend(sigmoid_v(output[0]))  #to have outputs between 0 and 1



In [14]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score,f1_score
all_values = []
for tensor in outputs_:
    all_values.append(tensor.item())

outputs__ = np.array(all_values) >= 0.5
accuracy = accuracy_score(targets_, outputs__)
precision = precision_score(targets_, outputs__)
recall = recall_score(targets_, outputs__)
f1_score = f1_score(targets_, outputs__) 
#average='micro'


print(f"Accuracy Score = {accuracy}")
print(f"Precision = {precision}")
print(f"Recall = {recall}")
print(f"F1 Score = {f1_score}")



Accuracy Score = 0.9528
Precision = 0.952
Recall = 0.9535256410256411
F1 Score = 0.9527622097678143
